In [ ]:
!pip uninstall -y hsfs
!pip install -U 'git+https://github.com/logicalclocks/feature-store-api@master#egg=hsfs[python]&subdirectory=python' --quiet
!pip uninstall -y hopsworks
!pip install -U 'git+https://github.com/logicalclocks/hopsworks-api@main#egg=hopsworks&subdirectory=python' --quiet

In [1]:
import re
import pandas as pd
import parsedatetime
from datetime import datetime
import numpy as np
import requests
import os

In [2]:
BACKFILL=False
version=1

In [3]:
if BACKFILL == False:
    filepath = 'index.html'
    URL = "https://www.lahinchsurfshop.com/"
    if os.path.exists(filepath):
        os.remove(filepath)
    response = requests.get(URL)
    open(filepath, "wb").write(response.content)    

    cal = parsedatetime.Calendar()
    data = []
    with open(filepath) as fp:
        line = fp.readline()
        cnt = 1
        the_date = None
        img_url = None
        max_height = None
        min_height = None
        regex = None
        while line:
            if "Last updated" in line:
                #: June 13, 2022
                str = line.partition("Last updated</b>:")[2]
                date_str = str.partition("</p>")[0]
                # fix the time to 10am
                date_str = date_str + " 10am"
                time_struct, parse_status = cal.parse(date_str)
                the_date = datetime(*time_struct[:6])
            if "Wave height" in line:
                str = line.partition("Wave height</b>:")[2]
                height_str = str.partition(", Sea")[0]
                regex = re.match(r'.*([0-9]+)-([0-9]+)ft.*', height_str)
                if regex is not None:
                    min_height=regex.groups()[0]
                    max_height=regex.groups()[1]
                    height = (float(min_height)+float(max_height))/2
                else:
                    regex = re.findall(r'.*([0-9]+)ft', height_str)
                    if len(regex):
                        max_height=regex[0]
                        min_height=regex[0]
                        height=regex[0]
                    else:
                        regex = re.findall(r'.*([4-9]+)inches', height_str)
                        if len(regex):
                            max_height="0.5"
                            min_height="0"
                            height="0.5"
                        else:
                            max_height="0"
                            min_height="0"
                            height="0"
                    
            if regex is not None:
                if the_date is not None:
                    row=[the_date, height, max_height, min_height]
                    data.append(row)
                    the_date = None
                    regex = None
            line = fp.readline()
            cnt += 1

    df = pd.DataFrame(data, columns=['observation_time', 'wave_height', 'max_height', 'min_height'])
    df['beach_id'] = 1
    df['wave_height'] = pd.to_numeric(df['wave_height'] , errors='coerce')
    df['max_height'] = pd.to_numeric(df['max_height'] , errors='coerce')
    df['min_height'] = pd.to_numeric(df['min_height'] , errors='coerce')
    
else: # BACKFILL == True
    df = pd.read_csv("data/lahinch-surf.csv",  parse_dates=['observation_time'])

df

,observation_time,wave_height,max_height,min_height,beach_id
0,2022-06-17 10:00:00,2.5,3,2,1


In [4]:
import great_expectations as ge

expectation_suite = ge.core.ExpectationSuite(expectation_suite_name="surf_height")

expectation_suite.add_expectation(
  ge.core.ExpectationConfiguration(
  expectation_type="expect_column_values_to_be_between",
  kwargs={"column":"wave_height", "min_value": 0, "max_value": 20}) 
);

In [5]:
import hopsworks
project = hopsworks.login()

Connected. Call `.close()` to terminate connection gracefully.

Logged in to project, explore it here https://c.app.hopsworks.ai:443/p/135


In [6]:
import hsfs

fs = project.get_feature_store()
lahinch_fg = fs.get_or_create_feature_group(name="lahinch",
        description="Lahinch beach surf height observations",
        online_enabled=True,
        version=version,
        primary_key=["beach_id"],
        expectation_suite=expectation_suite,
        statistics_config={"enabled": True, "histograms": True, "correlations": True},
        event_time="observation_time")

_, _ = lahinch_fg.insert(df)

Connected. Call `.close()` to terminate connection gracefully.
2022-06-18 07:36:23,008 INFO: 	1 expectation(s) included in expectation_suite.
Launching offline feature group backfill job...
Backfill Job started successfully, you can follow the progress at 
https://c.app.hopsworks.ai/p/135/jobs/named/lahinch_1_offline_fg_backfill/executions
